In [639]:
import pandas as pd

In [640]:
excel_file_path = "Dataset for People for their Blood Glucose Level with their Superficial body feature readings.csv.xlsx"

df = pd.read_excel(excel_file_path, sheet_name=0)



In [573]:
df.head(2)

,Age,Blood Glucose Level(BGL),Diastolic Blood Pressure,Systolic Blood Pressure,Heart Rate,Body Temperature,SPO2,Sweating (Y/N),Shivering (Y/N),Diabetic/NonDiabetic (D/N),Diabetic,Hypertension,HeartDisease,LungDisease
0,9,79,73,118,98,98.300707,99,0,0,N,0,False,False,False
1,9,80,73,119,102,98.300707,94,1,0,N,0,False,True,False


In [641]:
X = df[['Blood Glucose Level(BGL)', 'Diastolic Blood Pressure', 
        'Systolic Blood Pressure', 'Heart Rate', 'Body Temperature', 'SPO2']]

In [642]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [643]:
df['Diabetic'] = df['Diabetic/NonDiabetic (D/N)'].map({'D': 1, 'N': 0})
df['Hypertension'] = (df['Systolic Blood Pressure'] > 140) | (df['Diastolic Blood Pressure'] > 90)
df['HeartDisease'] = (df['Heart Rate'] > 100)  # 심박수 100 이상이면 위험
df['LungDisease'] = (df['SPO2'] < 92)  # 산소포화도 92% 미만이면 위험

In [644]:
df['Diabetic'] = label_encoder.fit_transform(df['Diabetic'])
df['Hypertension'] = label_encoder.fit_transform(df['Hypertension'])
df['HeartDisease'] = label_encoder.fit_transform(df['HeartDisease'])
df['LungDisease'] = label_encoder.fit_transform(df['LungDisease'])

In [ ]:
y = df[['Diabetic', 'Hypertension', 'HeartDisease', 'LungDisease']]

In [597]:
from sklearn.model_selection import train_test_split

In [598]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [599]:
from sklearn.preprocessing import StandardScaler

In [600]:
scaler=StandardScaler()

In [601]:
scaler.fit_transform(X)

array([[-0.38840755, -0.57634595, -0.02430669,  0.62210723,  1.16106334,
         1.90605148],
       [-0.36515279, -0.57634595,  0.10556114,  1.00637258,  1.16106334,
        -3.98556343],
       [-0.5977004 , -0.16205559, -1.06324937, -1.0110205 ,  1.16106334,
         0.7277285 ],
       ...,
       [-0.55119088,  0.66652513, -0.28404236,  0.14177555, -0.72506706,
         0.7277285 ],
       [-0.43491707,  1.21891228,  1.27437166, -0.05035713,  1.94816254,
         0.7277285 ],
       [-0.52793611,  1.08081549, -0.15417453, -1.20315317,  0.26371352,
        -0.45059449]])

In [602]:
y

,Diabetic,Hypertension,HeartDisease,LungDisease
0,0,0,0,0
1,0,0,1,0
2,0,0,0,0
3,0,0,0,0
4,0,1,0,0
...,...,...,...,...
16964,1,0,0,0
16965,1,0,0,0
16966,1,0,0,0
16967,1,0,0,0


In [566]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier


In [ ]:
regressor = LogisticRegression()
classifier=MultiOutputClassifier(regressor)


In [608]:
y_pred=classifier.predict_proba(X_test)

c:\Users\vhzkf\anaconda3\envs\hi\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [617]:
for i, disease in enumerate(y.columns):
    prob = y_pred[i][0][1] * 100 if len(y_pred[i][0]) > 1 else y_pred[i][0][0] * 100  # 확률을 퍼센트로 변환
    print(f"{disease}: {prob:.2f}%")

Diabetic: 9.64%
Hypertension: 54.86%
HeartDisease: 66.25%
LungDisease: 100.00%


ValueError: Input y contains NaN.

In [499]:
new_patient_data = pd.DataFrame([[80, 75, 120, 95, 37.0, 95]], 
                                columns=['Blood Glucose Level(BGL)', 'Diastolic Blood Pressure','Systolic Blood Pressure', 'Heart Rate', 'Body Temperature', 'SPO2'])

In [ ]:
import joblib

In [665]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd

# 데이터 준비
X = df[['Blood Glucose Level(BGL)', 'Diastolic Blood Pressure', 
        'Systolic Blood Pressure', 'Heart Rate', 'Body Temperature', 'SPO2']]

df['Diabetic'] = df['Diabetic/NonDiabetic (D/N)'].map({'D': 1, 'N': 0})
df['Hypertension'] = (df['Systolic Blood Pressure'] > 140) | (df['Diastolic Blood Pressure'] > 90)
df['HeartDisease'] = (df['Heart Rate'] > 100)  # 심박수 100 이상이면 위험

# Convert boolean columns to integers
df['Hypertension'] = df['Hypertension'].astype(int)
df['HeartDisease'] = df['HeartDisease'].astype(int)

y = df[['Diabetic', 'Hypertension', 'HeartDisease']]

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SMOTE 적용
smote = SMOTE(random_state=42)

# 각 타겟에 대해 SMOTE 적용
X_train_resampled_list = []
y_train_resampled_list = []

for i in range(y_train.shape[1]):
    X_res, y_res = smote.fit_resample(X_train_scaled, y_train.iloc[:, i])
    X_train_resampled_list.append(X_res)
    y_train_resampled_list.append(y_res)

# 공통된 X_train_resampled를 사용하고 y_train_resampled를 병합
X_train_resampled = X_train_resampled_list[0]
y_train_resampled = pd.concat([pd.Series(y_train_resampled_list[i]) for i in range(len(y_train_resampled_list))], axis=1)

# XGBoost 모델
xgb_model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)

# 다중 레이블 예측을 위해 MultiOutputClassifier 적용
multi_target_classifier = MultiOutputClassifier(xgb_model)

# 모델 학습
multi_target_classifier.fit(X_train_resampled, y_train_resampled)

# 교차검증
cross_val_results = cross_val_score(multi_target_classifier, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy: {cross_val_results.mean():.4f} ± {cross_val_results.std():.4f}")

# 예측 확률 계산
y_pred_proba = multi_target_classifier.predict_proba(X_test_scaled)

# 각 레이블에 대해 확률 출력
for i, disease in enumerate(y.columns):
    prob = y_pred_proba[i][0][1] * 100 if len(y_pred_proba[i][0]) > 1 else y_pred_proba[i][0][0] * 100  # 확률을 퍼센트로 변환
    print(f"{disease}: {prob:.2f}%")

# 평가: classification_report (이진 분류 문제로 평가)
y_pred = multi_target_classifier.predict(X_test_scaled)
print(classification_report(y_test, y_pred))


ValueError: Input y contains NaN.

In [ ]:
joblib.dump(classifier, 'health_modelAI.pkl')

['health_modelAI.pkl']